In [1]:
# Remember you need to use SWAN with NXCALS support....
from datetime import datetime
from datetime import timedelta
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector as msql
import pytimber
import os
ldb = pytimber.LoggingDB(spark_session = spark)

In [2]:
# pack daily data into text file     
# pickup signals       
BmExtrALPH  = 'LNE.APULB.0030:Acquisition:totalIntensitySingle' # Extr into 00 beam line (alpha,aegis,base ...
BmExtrGBAR  = 'LNE.APULB.5030:Acquisition:totalIntensitySingle' # Extr into 50 beam line (gbar, puma ...
SEM0008in   = 'LNE00.BSGW.0008:Acquisition:isMonitorIn' # SEM 0008 in
SEM0015in   = 'LNE00.BSGW.0015:Acquisition:isMonitorIn' # SEM 0015 in
SEM0025in   = 'LNE00.BSGW.0025:Acquisition:isMonitorIn' # SEM 0025 in
SEM5020in   = 'LNE50.BSGW.5020:Acquisition:isMonitorIn' # SEM 5020 in
SEM5060in   = 'LNE50.BSGW.5060:Acquisition:isMonitorIn' # SEM 5060 in
BmInjAD     = 'LNR.APULB.0227:Acquisition:totalIntensitySingle' # Inj into AD
CycleName   = 'ELENA.LSA:CYCLE'                         # Cycle name
LinacRM     = 'PARA611:RAW'                             # Radiation monitor Linac 
    
yr = 24
mt =  9

for dy in [20,21]:
    
    date = "20%02d-%02d-%02d " % (yr,mt,dy)                # human readable date
    fila = "ELENA_BM_%02d_%02d_%02d.csv" % (yr,mt,dy)      # filename for beam monitors
    filb = "ELENA_Cycles_%02d_%02d_%02d.csv" % (yr,mt,dy)  # filename for elena cycle
    filc = "Linac_RadMon_%02d_%02d_%02d.csv" % (yr,mt,dy)  # filename for linac rad monitor
    #fold = "/eos/user/l/lkoller/GBAR/data24/elenadata24/%02d_%02d_%02d" % (yr,mt,dy)
    fold = "/eos/experiment/gbar/elena/data/%02d_%02d_%02d" % (yr,mt,dy)

    if not os.path.exists(fold):
        os.makedirs(fold)    # includes creation of all folders in path
    
    fdata = os.path.join(fold,fila)    #    fdata = fold + "/" + fila
    fcycl = os.path.join(fold,filb)    #    fcycl = fold + "/" + filb 
    fradm = os.path.join(fold,filc)    #    fradm = fold + "/" + filc 

#    if os.path.isfile(fdata) and os.path.isfile(fcycl):
    if 1==2:
        print(date, 'data and cycle files exist - nothing to be done')
    else:
        t1 = date + '00:00:00.000'
        t2 = date + '23:59:59.999'
        print(t1)
        dat1 = ldb.get([BmExtrALPH,BmExtrGBAR],t1,t2)
        dat2 = ldb.get([SEM0008in,SEM0015in,SEM0025in,SEM5020in,SEM5060in],t1,t2)
        dat3 = ldb.get([BmInjAD],t1,t2)
        dat = {**dat1,**dat2,**dat3}
        kys  = list(dat)                       #print(kys)

        if kys:                                #print("we found some data")    
            cyc  = ldb.get([CycleName],t1,t2)
            kyc  = list(cyc)                   # print(kyc)  # print(cyc[kyc[0]][0][-1])
            
            lrm  = ldb.get([LinacRM],t1,t2)
            krm  = list(lrm)                   # print(krm)  # print(lrm[krm[0]][0][-1])
            
            evs  = len(dat[kys[0]][0])
            evc  = len(cyc[kyc[0]][0])
            evr  = len(lrm[krm[0]][0])
            print(date,'found ',evs,' extraction, ',evr,' rad mon entries and ',evc,' cycle entries')

            fil = open(fdata, "w")             # write a comma separated data file 
            # header
            fil.write("%d entries: Unix_Time,%s,%s,%s,%s,%s,%s,%s,%s\n" % (evs,BmExtrALPH,BmExtrGBAR,
                                               SEM0008in,SEM0015in,SEM0025in,SEM5020in,SEM5060in,BmInjAD))            
            for j in range(evs):
                good=True
                try:
                    dataline = (dat[kys[1]][0][j],dat[kys[0]][1][j],dat[kys[1]][1][j], # first is timestamp from extr to GBAR
                                dat[kys[2]][1][j],dat[kys[3]][1][j],dat[kys[4]][1][j],
                                dat[kys[5]][1][j],dat[kys[6]][1][j],dat[kys[7]][1][j] ) 
                except IndexError:                                        #print('Test index ',j,' dat ', dat[kys[6]][1][j] ) 
                    good=False
                    print('Index error in event ',j,' skip event')

                if good:
                    fil.write("%.3f,%.3f,%.3f,%d,%d,%d,%d,%d,%.3f\n" % (dataline) )
                    #fil.write("%.3f,%.3f,%.3f," % (dat[kys[1]][0][j],dat[kys[0]][1][j],dat[kys[1]][1][j]) )
                    #fil.write("%d,%d,%d,"       % (dat[kys[2]][1][j],dat[kys[3]][1][j],dat[kys[4]][1][j]) )
                    #fil.write("%d,%d,%.3f\n"    % (dat[kys[5]][1][j],dat[kys[6]][1][j],dat[kys[7]][1][j]) )        
            fil.close()

            if krm:
                fil = open(fradm, "w") # write a comma separated file with time stamps and the radiation monitor 
                fil.write("%d entries: Unix_Time,%s\n" % (evr,LinacRM)) # header           
                for j in range(evr):
                    #if cyc[kyc[0]][1][j] != 'ELENA_ZERO_CYCLE' :
                    fil.write("%.3f,%.8f\n" % (lrm[krm[0]][0][j],lrm[krm[0]][1][j]))        
                fil.close()
            else:
                print(date, "no rad mon info")            
            
            if kyc:
                fil = open(fcycl, "w") # write a comma separated file with time stamps and the Elena cycles 
                fil.write("%d entries: Unix_Time,%s\n" % (evc,CycleName)) # header                
                for j in range(evc):
                    if cyc[kyc[0]][1][j] != 'ELENA_ZERO_CYCLE' :
                        fil.write("%.3f,%s\n" % (cyc[kyc[0]][0][j],cyc[kyc[0]][1][j]))        
                fil.close()                  
            else:
                print(date, "no cycle info")
                  
        else:
            os.rmdir(fold) 
            print(date, "no run data, folder '%s' has been removed " % fold)
            
        print(date)
            

2024-09-20 00:00:00.000
2024-09-20  found  4646  extraction,  71988  rad mon entries and  21452  cycle entries
2024-09-20 
2024-09-21 00:00:00.000
2024-09-21  found  2582  extraction,  29344  rad mon entries and  7702  cycle entries
2024-09-21 
